이전 자료에서 다루었던 추천 시스템을 실습으로 살펴봅니다.

해당 자료는 아래 리스트에서 참고했습니다.

- https://www.kaggle.com/rounakbanik/movie-recommender-systems
- https://www.kaggle.com/ibtesama/getting-started-with-a-movie-recommendation-system


데이터는 kaggle의 **The movies Dataset (https://www.kaggle.com/rounakbanik/the-movies-dataset)** 을 사용했습니다.


가장 먼저 데이터 전처리와 콘텐츠 기반(content based filtering)으로 시작합니다.

# 데이터 전처리

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

데이터를 불러옵니다.

In [2]:
data = pd.read_csv('./movie_data/tmdb_5000_movies.csv')

In [3]:
data.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [4]:
data.shape

(4803, 20)

데이터에 대한 설명은 참조 링크에 들어가면 자세히 쓰여져 있습니다.  
여기서 제가 일반적으로 많이 사용하는 컬럼은 아래와 같습니다.  

- genres : 영화 장르
- keywords : 영화의 키워드
- original_language : 영화 언어 
- title : 제목
- vote_average : 평점 평균
- vote_count :  평점 카운트
- popularity : 인기도
- overview : 개요 설명

등등 같은 컬럼을 사용할 예정입니다. 다른 컬럼은 일단 여기서 그렇게 중요하지 않게 사용합니다.  
사실, release_data와 같은 컬럼도 중요할 수 있습니다. 최신 영화를 추천할 수도 있으니까요. 하지만 여기서는 사용하지 않겠습니다.  


가장 먼저 **전처리**를 조금 해주어야 합니다.  
먼저 우리가 사용할 데이터부터 뽑아보죠.


In [5]:
data = data[['id','genres', 'vote_average', 'vote_count','popularity','title',  'keywords', 'overview']]


그리고 vote_average값을 변경해주어야 합니다.   
현재 vote_average는 조금 **불공정**하게 되어 있습니다.

왜냐하면, vote 수가 적은데(예를 들어 3개) 3개 전부 5점이라고 하면 vote가 5점으로 되어 있기 때문입니다.  
하지만, vote 수가 많을수록 vote_average가 떨어질 수 밖에 없습니다. 많은 사람들이 평가를 하니까요.  

그래서 이런 불공정을 처리하기 위해 imdb에서 처리한 방법이 있습니다.  
해당 이슈는 url : https://www.quora.com/How-does-IMDbs-rating-system-work 에서 확인할 수 있습니다.

그에 대한 답은 아래와 같습니다.

![1](https://user-images.githubusercontent.com/24634054/71774470-d1470c80-2fb2-11ea-8a1e-aa018dd6d25a.JPG)

- r : 개별 영화 평점
- v : 개별 영화에 평점을 투표한 횟수
- m : 250위 안에 들어야 하는 최소 투표 (정하기 나름인듯. 난 500이라고 하면 500으로 해도 되고.)
- c : 전체 영화에 대한 평균 평점

여기서 m은 **500위로 가정하고 진행하겠습니다.** 

먼저 m부터 찾아보죠. 500위 정도로 들어오게 하려면 vote_count가 상위 몇 %이어야 할까요?  
이는 quantile을 이용해서 구할 수 있습니다.

In [6]:
tmp_m = data['vote_count'].quantile(0.89)
tmp_m

1683.8999999999987

In [7]:
tmp_data = data.copy().loc[data['vote_count'] >= tmp_m]
tmp_data.shape

(529, 8)

상위 90%로 했을 때 481개가 들어옵니다.   
89%로 하면 529개가 들어오게 됩니다. 저는 90%로 가정하고 진행하도록 하겠습니다.

In [8]:
del tmp_data

m = data['vote_count'].quantile(0.9)
data = data.loc[data['vote_count'] >= m]

In [9]:
data.head()

,id,genres,vote_average,vote_count,popularity,title,keywords,overview
0,19995,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",7.2,11800,150.437577,Avatar,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di..."
1,285,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",6.9,4500,139.082615,Pirates of the Caribbean: At World's End,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha..."
2,206647,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",6.3,4466,107.376788,Spectre,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...
3,49026,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",7.6,9106,112.312950,The Dark Knight Rises,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...
4,49529,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",6.1,2124,43.926995,John Carter,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca..."


In [10]:
C = data['vote_average'].mean()

In [11]:
print(C)
print(m)

6.962993762993763
1838.4000000000015


In [12]:
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    
    return ( v / (v+m) * R ) + (m / (m + v) * C)

In [13]:
data['score'] = data.apply(weighted_rating, axis = 1)

In [14]:
data.head(5)

,id,genres,vote_average,vote_count,popularity,title,keywords,overview,score
0,19995,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",7.2,11800,150.437577,Avatar,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",7.168053
1,285,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",6.9,4500,139.082615,Pirates of the Caribbean: At World's End,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha...",6.918271
2,206647,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",6.3,4466,107.376788,Spectre,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...,6.493333
3,49026,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",7.6,9106,112.312950,The Dark Knight Rises,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...,7.492998
4,49529,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",6.1,2124,43.926995,John Carter,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca...",6.500396


In [15]:
data.shape

(481, 9)

이렇게 weighted_score가 완성되었습니다.

또한, 지금 장르와 키워드를 보시면 조금 독특한 구조의 데이터를 가지고 있습니다.

In [16]:
data[['genres', 'keywords']].head(2)

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na..."


list 내부에 dictionary가 있는 구조로 되어있습니다.  
이렇게 표현한 이유는 하나의 영화가 하나의 장르에만 속하지 않고, 하나의 키워드만 있지 않기 때문입니다.  
그리고 문제가 지금 내부에는 **문자열**로 들어가 있는 것입니다.

이를 해결하기 위해서 ast 패키지를 사용해야합니다. ast내부에 literal_eval을 사용해보죠  

그러면 list와 dictionary 형태로 바뀌게 됩니다.

In [17]:
data['genres'] = data['genres'].apply(literal_eval)
data['keywords'] = data['keywords'].apply(literal_eval)

In [18]:
data[['genres', 'keywords']].head(2)

,genres,keywords
0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...","[{'id': 1463, 'name': 'culture clash'}, {'id':..."
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'id': 270, 'name': 'ocean'}, {'id': 726, 'na..."


그럼 이제 장르와 키워드를 id를 제거한 후 name만 뽑아내면 끝나겠죠?  
우리에게 저 id 값을 필요없으니까요. 장르가 무엇이고 키워드가 무엇인지만 알면됩니다.

In [19]:
data['genres'] = data['genres'].apply(lambda x : [d['name'] for d in x]).apply(lambda x : " ".join(x))
data['keywords'] = data['keywords'].apply(lambda x : [d['name'] for d in x]).apply(lambda x : " ".join(x))

In [20]:
data.head(2)

,id,genres,vote_average,vote_count,popularity,title,keywords,overview,score
0,19995,Action Adventure Fantasy Science Fiction,7.2,11800,150.437577,Avatar,culture clash future space war space colony so...,"In the 22nd century, a paraplegic Marine is di...",7.168053
1,285,Adventure Fantasy Action,6.9,4500,139.082615,Pirates of the Caribbean: At World's End,ocean drug abuse exotic island east india trad...,"Captain Barbossa, long believed to be dead, ha...",6.918271


다음장에서 사용할 데이터이므로 미리 저장을 합니다.

In [21]:
data.to_csv('./movie_data/pre_tmdb_5000_movies.csv', index = False)

자! 이렇게 어느정도 전처리가 마무리 되었습니다.

이제 본격적으로 진행해보죠.

# 콘텐츠 기반 필터링 추천(Content based filtering)

콘텐츠 기반으로 추천을 하고자 합니다. 콘텐츠 기반 필터링을 이용해서 추천을 진행하는 것은 비슷한 콘텐츠를 사용자에게 추천하는 것을 말합니다.  

여기서 비슷한 콘텐츠는 무엇일까요? 대표적으로 '장르'가 될 수 있습니다.   
따라서 content based filtering 추천에서는 이 '장르'를 이용해서 추천을 하겠습니다.  

현재 장르는 문자열로 되어 있습니다. 이 문자열을 숫자로 바꾸어 벡터화 시켜야겠죠? 이것부터 진행하죠

In [22]:
data.genres.head(2)

0    Action Adventure Fantasy Science Fiction
1                    Adventure Fantasy Action
Name: genres, dtype: object

In [23]:
count_vector = CountVectorizer(ngram_range=(1, 3))

In [24]:
c_vector_genres = count_vector.fit_transform(data['genres'])


In [25]:
c_vector_genres.shape

(481, 364)

이렇게 하면 단어를 벡터화 시켜서 저장할 수 있습니다. 

이제 각 영화의 유사도를 측정을 하면됩니다. 유사도를 측정하면 장르가 비슷한 영화가 추천되겠죠?  
이 유사도 측정은 코사인 유사도(cosine similarity)를 사용합니다.

자! 그리고 함수를 하나 만들겁니다. 이 함수의 기능은 아래와 같습니다.  
1. 코사인 유사도를 이용해 장르가 비슷한 영화를 추천
2. vote_count를 이용해서 vote_count가 높은 것을 기반으로 최종 추천


In [26]:
#코사인 유사도를 구한 벡터를 미리 저장
gerne_c_sim = cosine_similarity(c_vector_genres, c_vector_genres).argsort()[:, ::-1]

In [27]:
gerne_c_sim.shape

(481, 481)

In [28]:
def get_recommend_movie_list(df, movie_title, top=30):
    # 특정 영화와 비슷한 영화를 추천해야 하기 때문에 '특정 영화' 정보를 뽑아낸다.
    target_movie_index = df[df['title'] == movie_title].index.values
    
    #코사인 유사도 중 비슷한 코사인 유사도를 가진 정보를 뽑아낸다.
    sim_index = gerne_c_sim[target_movie_index, :top].reshape(-1)
    #본인을 제외
    sim_index = sim_index[sim_index != target_movie_index]

    #data frame으로 만들고 vote_count으로 정렬한 뒤 return
    result = df.iloc[sim_index].sort_values('score', ascending=False)[:10]
    return result
    
    
    

In [29]:
get_recommend_movie_list(data, movie_title='The Dark Knight Rises')

,id,genres,vote_average,vote_count,popularity,title,keywords,overview,score
65,155,Drama Action Crime Thriller,8.2,12002,187.322927,The Dark Knight,dc comics crime fighter secret identity scarec...,Batman raises the stakes in his war on crime. ...,8.035690
2091,274,Crime Drama Thriller,8.1,4443,18.174804,The Silence of the Lambs,based on novel psychopath horror suspense seri...,"FBI trainee, Clarice Starling ventures into a ...",7.767228
2760,264644,Drama Thriller,8.1,2757,66.113340,Room,based on novel carpet isolation kidnapping imp...,Jack is a young boy of 5 years old who has liv...,7.645138
351,1422,Drama Thriller Crime,7.9,4339,63.429157,The Departed,undercover boston police friends mafia underco...,"To take down South Boston's Irish Mafia, the p...",7.621146
1850,111,Action Crime Drama Thriller,8.0,2948,70.105981,Scarface,miami corruption capitalism cuba prohibition b...,After getting a green card in exchange for ass...,7.601698
4337,103,Crime Drama,8.0,2535,58.845025,Taxi Driver,vietnam veteran taxi obsession drug dealer nig...,A mentally unstable Vietnam War veteran works ...,7.564085
1051,146233,Drama Thriller Crime,7.9,3085,88.496873,Prisoners,pennsylvania kidnapping maze vigilante rural s...,When Keller Dover's daughter and her friend go...,7.550121
828,24,Action Crime,7.7,4949,79.754966,Kill Bill: Vol. 1,japan coma martial arts kung fu underworld yak...,An assassin is shot at the altar by her ruthle...,7.500378
3701,641,Crime Drama,7.9,2443,11.573034,Requiem for a Dream,drug addiction junkie heroin speed diet unsoci...,The hopes and dreams of four ambitious people ...,7.497657
1829,6977,Crime Drama Thriller,7.7,3003,53.645267,No Country for Old Men,texas drug traffic hitman united states–mexico...,"Llewelyn Moss stumbles upon dead bodies, $2 mi...",7.420140


이렇게 하면 The Dark Knight Rises와 비슷한 영화가 content based filtering 방법으로 추천이 됩니다.

The Dark Knight Rise 영화의 장르는 Action, Crime, Drama, Thriller 종류입니다. 추천된 종류 역시 이와 같이 비슷한 장르의 특성을 보여주고 있음을 알 수 있습니다.


In [30]:
data[data['title'] == 'The Dark Knight Rises']

,id,genres,vote_average,vote_count,popularity,title,keywords,overview,score
3,49026,Action Crime Drama Thriller,7.6,9106,112.31295,The Dark Knight Rises,dc comics crime fighter terrorist secret ident...,Following the death of District Attorney Harve...,7.492998
